In [1]:
import cv2
import numpy as np
from ultralytics import YOLO

print("OpenCV Version:", cv2.__version__)
print("NumPy Version:", np.__version__)
print("Ultralytics YOLO Imported Successfully!")


f:\Python\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Admin\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
OpenCV Version: 4.10.0
NumPy Version: 1.26.4
Ultralytics YOLO Imported Successfully!


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Load YOLO model
model = YOLO("yolov8n.pt")

# Initialize video capture (0 for webcam, or replace with video file path)
cap = cv2.VideoCapture(0)

# Store detected vehicles (ID -> (x, y, type))
tracked_vehicles = {}
vehicle_id_counter = 0

# Active vehicle count per type
active_vehicles = defaultdict(int)

def get_vehicle_counts():
    """Returns the current count of detected vehicles dynamically."""
    return {k: v for k, v in active_vehicles.items() if v > 0}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection
    results = model(frame)

    # Temporary storage for new vehicle positions
    new_tracked_vehicles = {}

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            class_id = int(box.cls[0])
            class_name = model.names[class_id]

            # Only consider vehicle types
            if class_name in {"car", "truck", "bus", "motorcycle"}:

                # Check if this vehicle is already tracked
                assigned_id = next(
                    (vid for vid, (px, py, v_type) in tracked_vehicles.items()
                     if abs(center_x - px) < 50 and abs(center_y - py) < 50), 
                    None
                )

                # Assign new ID if it's a new vehicle
                if assigned_id is None:
                    assigned_id = vehicle_id_counter
                    vehicle_id_counter += 1
                    active_vehicles[class_name] += 1

                # Store new position
                new_tracked_vehicles[assigned_id] = (center_x, center_y, class_name)

                # Draw bounding box & label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {assigned_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.circle(frame, (center_x, center_y), 5, (255, 255, 255), -1)

    # Update vehicle counts for exiting vehicles
    for vid, (_, _, v_type) in tracked_vehicles.items():
        if vid not in new_tracked_vehicles:
            active_vehicles[v_type] = max(0, active_vehicles[v_type] - 1)

    # Update tracked vehicles
    tracked_vehicles = new_tracked_vehicles

    # Display vehicle counts on screen
    for i, (v_type, count) in enumerate(get_vehicle_counts().items()):
        cv2.putText(frame, f"{v_type}: {count}", (30, 50 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show frame with detections
    cv2.imshow("Vehicle Detection", frame)

    # Print the latest vehicle count dynamically
    print(get_vehicle_counts())

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Final count after video ends
print("Final Count:", get_vehicle_counts())


f:\Python\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(



0: 480x640 1 person, 117.8ms
Speed: 3.4ms preprocess, 117.8ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 1 bottle, 104.2ms
Speed: 3.0ms preprocess, 104.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 106.2ms
Speed: 3.4ms preprocess, 106.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 170.4ms
Speed: 4.3ms preprocess, 170.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 103.0ms
Speed: 1.4ms preprocess, 103.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 150.8ms
Speed: 2.2ms preprocess, 150.8ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 128.2ms
Speed: 1.9ms preprocess, 128.2ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
{}

0: 480x640 1 person, 106.9ms
Speed: 2.3ms preprocess, 106.9ms inference,

KeyboardInterrupt: 